<a href="https://colab.research.google.com/github/darshan-stack/Auto-R/blob/main/Multithreading_concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import time
import threading
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime

# just some colors to make output easier to read
class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    ORANGE = '\033[93m'
    RED = '\033[91m'
    END = '\033[0m'
    BOLD = '\033[1m'

## Simulating LLM Gateway Operations

Testing out different scenarios:
- Slow LLM API Call (GPT-4 style) - about 3 seconds
- Fast Cache Hit (Redis) - 0.5 seconds
- Database Query (budget check) - 1 second

In [18]:
def customer_a_llm_request(customer_id):
    # simulating a slow GPT-4 call
    start = time.time()
    print(f"{Colors.BLUE} Customer A: Starting GPT-4 API call...{Colors.END}")

    time.sleep(3)  # API is slow

    end = time.time()
    result = f"GPT-4 Response: 'Hello, I can help you!'"
    print(f"{Colors.BLUE} Customer A: Got response! Took {end-start:.2f}s{Colors.END}")
    return result

def customer_b_cache_hit(customer_id):
    # this should be fast - just reading from cache
    start = time.time()
    print(f"{Colors.GREEN} Customer B: Checking cache...{Colors.END}")

    time.sleep(0.5)  # cache lookup

    end = time.time()
    result = f"Cached Response: 'AI is Artificial Intelligence'"
    print(f"{Colors.GREEN} Customer B: Cache hit! {end-start:.2f}s{Colors.END}")
    return result

def customer_c_budget_check(customer_id):
    # checking database for budget
    start = time.time()
    print(f"{Colors.ORANGE} Customer C: Running budget query...{Colors.END}")

    time.sleep(1)  # db query

    end = time.time()
    result = f"Budget OK: User has $50 remaining"
    print(f"{Colors.ORANGE} Customer C: Done. {end-start:.2f}s{Colors.END}")
    return result

## Method 1: Single-threaded approach

The problem here: everything runs one after another
- Customer B has to wait for A to finish
- Customer C waits for both A and B
- Total time adds up

In [19]:
print(f"{Colors.BOLD}{'='*60}{Colors.END}")
print(f"{Colors.BOLD}SINGLE-THREADED (sequential processing){Colors.END}")
print(f"{Colors.BOLD}{'='*60}{Colors.END}\n")

start_total = time.time()

# just running them one by one
result_a = customer_a_llm_request("A")
result_b = customer_b_cache_hit("B")
result_c = customer_c_budget_check("C")

end_total = time.time()
total_time = end_total - start_total

print(f"\n{Colors.RED}{Colors.BOLD}Total time: {total_time:.2f} seconds{Colors.END}")
print(f"{Colors.RED}Customer B waited {total_time:.2f}s even though their request only needed 0.5s{Colors.END}")
print(f"{Colors.RED}Customer C also waited the full {total_time:.2f}s{Colors.END}\n")

SINGLE-THREADED (sequential processing)

 Customer A: Starting GPT-4 API call...
 Customer A: Got response! Took 3.00s
 Customer B: Checking cache...
 Customer B: Cache hit! 0.50s
 Customer C: Running budget query...
 Customer C: Done. 1.00s

Total time: 4.50 seconds
Customer B waited 4.50s even though their request only needed 0.5s
Customer C also waited the full 4.50s



## Method 2: Multi-threaded

Now let's try running everything at the same time
- All customers get served simultaneously
- Total time should be roughly the slowest operation
- Much better!

In [20]:
print(f"{Colors.BOLD}{'='*60}{Colors.END}")
print(f"{Colors.BOLD}MULTI-THREADED (parallel processing){Colors.END}")
print(f"{Colors.BOLD}{'='*60}{Colors.END}\n")

start_total = time.time()

# using ThreadPoolExecutor to run them all at once
with ThreadPoolExecutor(max_workers=3) as executor:
    # submit all tasks
    future_a = executor.submit(customer_a_llm_request, "A")
    future_b = executor.submit(customer_b_cache_hit, "B")
    future_c = executor.submit(customer_c_budget_check, "C")

    # get results
    result_a = future_a.result()
    result_b = future_b.result()
    result_c = future_c.result()

end_total = time.time()
total_time = end_total - start_total

print(f"\n{Colors.GREEN}{Colors.BOLD}Total time: {total_time:.2f} seconds{Colors.END}")
print(f"{Colors.GREEN}All customers served in {total_time:.2f}s (just the slowest operation){Colors.END}")
print(f"{Colors.GREEN}Customer B finished in 0.5s and didn't wait{Colors.END}")
print(f"{Colors.GREEN}Customer C finished in 1s and didn't wait{Colors.END}\n")

MULTI-THREADED (parallel processing)

 Customer A: Starting GPT-4 API call...
 Customer B: Checking cache...
 Customer C: Running budget query...
 Customer B: Cache hit! 0.50s
 Customer C: Done. 1.00s
 Customer A: Got response! Took 3.00s

Total time: 3.00 seconds
All customers served in 3.00s (just the slowest operation)
Customer B finished in 0.5s and didn't wait
Customer C finished in 1s and didn't wait



## Performance



In [21]:
# let's measure both approaches properly
import time

# single threaded - everything adds up
start = time.time()
time.sleep(3)  # LLM call
time.sleep(0.5)  # Cache hit
time.sleep(1)  # Budget check
single_threaded_time = time.time() - start

# multi threaded - just the slowest one matters
multi_threaded_time = max(3, 0.5, 1)

print(f"{Colors.BOLD}{'='*60}{Colors.END}")
print(f"{Colors.BOLD}COMPARISON{Colors.END}")
print(f"{Colors.BOLD}{'='*60}{Colors.END}\n")

print(f"Single-threaded: {single_threaded_time:.2f}s")
print(f"Multi-threaded:  {multi_threaded_time:.2f}s")
print(f"\n{Colors.GREEN}Speed improvement: {(single_threaded_time/multi_threaded_time):.2f}x faster{Colors.END}")
print(f"{Colors.GREEN}Can handle {(single_threaded_time/multi_threaded_time):.2f}x more customers{Colors.END}\n")

COMPARISON

Single-threaded: 4.50s
Multi-threaded:  3.00s

Speed improvement: 1.50x faster
Can handle 1.50x more customers



## Real scenario - handling 10 customers

Let's try something more realistic with 10 different requests:
- 4 slow LLM calls
- 4 fast cache hits
- 2 budget checks


In [22]:
def process_request(request_id, request_type):
    # handle different request types
    start = time.time()

    if request_type == "llm":
        print(f" Request {request_id}: Calling LLM API...")
        time.sleep(3)
        result = "LLM Response"
    elif request_type == "cache":
        print(f" Request {request_id}: Cache Hit!")
        time.sleep(0.5)
        result = "Cached Data"
    else:  # budget check
        print(f" Request {request_id}: Budget Check...")
        time.sleep(1)
        result = "Budget OK"

    end = time.time()
    print(f" Request {request_id} done in {end-start:.2f}s")
    return result

# 10 different customer requests
requests = [
    (1, "llm"), (2, "cache"), (3, "llm"), (4, "cache"),
    (5, "budget"), (6, "llm"), (7, "cache"), (8, "llm"),
    (9, "cache"), (10, "budget")
]

### Multi-threaded: everyone gets served at once

In [23]:
print(f"{Colors.BOLD}{'='*60}{Colors.END}")
print(f"{Colors.BOLD}MULTI-THREADED: all 10 customers at once{Colors.END}")
print(f"{Colors.BOLD}{'='*60}{Colors.END}\n")

start_time = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_request, req_id, req_type)
               for req_id, req_type in requests]

    results = [future.result() for future in futures]

multi_total = time.time() - start_time
print(f"\n{Colors.GREEN}{Colors.BOLD}Total time: {multi_total:.2f} seconds{Colors.END}")
print(f"{Colors.GREEN}All 10 customers done in {multi_total:.2f}s!{Colors.END}\n")

MULTI-THREADED: all 10 customers at once

 Request 1: Calling LLM API...
 Request 2: Cache Hit!
 Request 3: Calling LLM API...
 Request 4: Cache Hit!
 Request 5: Budget Check...
 Request 6: Calling LLM API...
 Request 7: Cache Hit!
 Request 8: Calling LLM API...
 Request 9: Cache Hit!
 Request 10: Budget Check...
 Request 2 done in 0.50s
 Request 4 done in 0.50s
 Request 7 done in 0.50s
 Request 9 done in 0.50s
 Request 5 done in 1.00s
 Request 10 done in 1.00s
 Request 1 done in 3.00s
 Request 3 done in 3.00s
 Request 6 done in 3.00s
 Request 8 done in 3.00s

Total time: 3.01 seconds
All 10 customers done in 3.01s!



## Final comparison


In [24]:
print(f"{Colors.BOLD}{'='*70}{Colors.END}")
print(f"{Colors.BOLD}          MULTITHREADING RESULTS          {Colors.END}")
print(f"{Colors.BOLD}{'='*70}{Colors.END}\n")

print(f"Scenario: 10 customers (4 LLM, 4 cache, 2 DB)\n")

# calculating expected times
single_expected = (4 * 3) + (4 * 0.5) + (2 * 1)  # everything adds up
multi_expected = 3  # just the slowest one

print(f"{Colors.RED}Single-threaded:{Colors.END}")
print(f"   Time: {single_expected:.1f}s")
print(f"   (everyone waits in line)\n")

print(f"{Colors.GREEN}Multi-threaded:{Colors.END}")
print(f"   Time: ~{multi_expected:.1f}s")
print(f"   (all processed simultaneously)\n")

speedup = single_expected / multi_expected
print(f"{Colors.BOLD}{Colors.GREEN}Performance: {speedup:.1f}x faster{Colors.END}")
print(f"{Colors.BOLD}{Colors.GREEN}Throughput: {speedup:.1f}x more customers/sec{Colors.END}")
print(f"{Colors.BOLD}{Colors.GREEN}Much better user experience\n{Colors.END}")

print(f"{Colors.BOLD}{'='*70}{Colors.END}")
print(f"{Colors.BOLD}This is basically how FastAPI handles requests{Colors.END}")
print(f"{Colors.BOLD}{'='*70}{Colors.END}")

          MULTITHREADING RESULTS          

Scenario: 10 customers (4 LLM, 4 cache, 2 DB)

Single-threaded:
   Time: 16.0s
   (everyone waits in line)

Multi-threaded:
   Time: ~3.0s
   (all processed simultaneously)

Performance: 5.3x faster
Throughput: 5.3x more customers/sec
Much better user experience

This is basically how FastAPI handles requests
